----
<img src="../../../files/refinitiv.png" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Delivery layer - OMM Stream - Level 2 orderbook subscription
This notebook demonstrates how to use an OMM stream of the low level Delivery layer to subscribe to retrieve level 2 orderbooks.

#### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the Refinitiv Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/learning), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/docs)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help. 

## Set the configuration file location
For a better ease of use, you have the option to set initialization parameters of the Refinitiv Data Library in the _refinitiv-data.config.json_ configuration file. This file must be located beside your notebook, in your user folder or in a folder defined by the _RD_LIB_CONFIG_PATH_ environment variable. The _RD_LIB_CONFIG_PATH_ environment variable is the option used by this series of examples. The following code sets this environment variable.      

In [1]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../../Configuration"

## Some Imports to start with

In [2]:
import refinitiv.data as rd
from refinitiv.data.delivery import omm_stream
import datetime
import json

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [3]:
rd.open_session("platform.deployed")

<refinitiv.data.session.platform.Definition object at 0x7faead4616a0 {session_name='deployed-cipsnylab2'}>

## Retrieve data

### Define a calback to display received events

In [4]:
def display_event(eventType, event):
    currentTime = datetime.datetime.now().time()
    print("----------------------------------------------------------")
    print(">>> {} event received at {}".format(eventType, currentTime))
    print(json.dumps(event, indent=2))
    return

### Create an OMM stream and register event callback

In [5]:
stream = omm_stream.Definition(
    domain="MarketByPrice", 
    name="BB.TO",
    service="ELEKTRON_DD"
).get_stream()

stream.on_refresh(lambda event, item_stream : display_event("Refresh", event))
stream.on_update(lambda event, item_stream : display_event("Update", event))
stream.on_status(lambda event, item_stream : display_event("Status", event))
stream.on_error(lambda event, item_stream : display_event("Error", event))

### Open the stream
This example uses the open() synchronous call to open the stream. Because it is a synchronous call, the first notification (either via on_refresh(), on_status() or on_error()) happens before the open() method returns. If the open_async() asynchronous method is used instead, the first notification happens after open_async() returns.

In [6]:
stream.open()

----------------------------------------------------------
>>> Refresh event received at 15:59:19.980556
{
  "ID": 5,
  "Type": "Refresh",
  "Domain": "MarketByPrice",
  "Key": {
    "Service": "ELEKTRON_DD",
    "Name": "BB.TO"
  },
  "State": {
    "Stream": "Open",
    "Data": "Ok",
    "Text": "All is well"
  },
  "Complete": false,
  "Qos": {
    "Timeliness": "Realtime",
    "Rate": "TickByTick"
  },
  "PartNumber": 0,
  "PermData": "AwFkMETA",
  "SeqNumber": 30688,
  "Map": {
    "KeyType": "Buffer",
    "Summary": {
      "Fields": {
        "PROD_PERM": 3044,
        "DSPLY_NAME": "BLACKBERRY LTD",
        "CURRENCY": "CAD",
        "ACTIV_DATE": "2022-06-22",
        "PRC_QL2": " AU",
        "LOT_SIZE_A": 100,
        "RECORDTYPE": 113,
        "PREF_DISP": null,
        "RDN_EXCHD2": "TOR",
        "LIST_MKT": "TOR",
        "PROV_SYMB": "BB",
        "PR_RNK_RUL": "NOR",
        "OR_RNK_RUL": "PTS ",
        "MNEMONIC": "BB",
        "MKT_STATUS": "S",
        "TIMACT_MS":

<OpenState.Opened: 'Opened'>

----------------------------------------------------------
>>> Update event received at 15:59:20.454904
{
  "ID": 5,
  "Type": "Update",
  "Domain": "MarketByPrice",
  "UpdateType": "Unspecified",
  "Key": {
    "Service": "ELEKTRON_DD",
    "Name": "BB.TO"
  },
  "SeqNumber": 30720,
  "Map": {
    "KeyType": "Buffer",
    "Summary": {
      "Fields": {
        "TIMACT_MS": 50360435
      }
    },
    "Entries": [
      {
        "Action": "Update",
        "Key": "Ni45MjAwMDBC",
        "Fields": {
          "ORDER_PRC": 6.92,
          "ORDER_SIDE": "BID",
          "ACC_SIZE": 5900,
          "NO_ORD": 7,
          "LV_TIM_MS": 50346430,
          "LV_TIM_MSP": 712,
          "LV_DATE": "2022-06-22"
        }
      }
    ]
  }
}
----------------------------------------------------------
>>> Update event received at 15:59:20.456105
{
  "ID": 5,
  "Type": "Update",
  "Domain": "MarketByPrice",
  "UpdateType": "Unspecified",
  "Key": {
    "Service": "ELEKTRON_DD",
    "Name": "BB.TO"


### Close the stream

In [7]:
stream.close()

<OpenState.Closed: 'Closed'>

## Close the session

In [8]:
rd.close_session()